In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle

In [4]:
reads = pd.read_csv('TCGA-BRCA.htseq_fpkm.tsv', sep='\t', header=0, index_col=0)

# reads.rename(columns={'index':'Sample'}, inplace=True)
reads

,TCGA-E9-A1NI-01A,TCGA-A1-A0SP-01A,TCGA-BH-A1EU-11A,TCGA-A8-A06X-01A,TCGA-E2-A14T-01A,TCGA-AC-A8OS-01A,TCGA-A8-A09K-01A,TCGA-OL-A5RY-01A,TCGA-BH-A0DG-01A,TCGA-B6-A0I9-01A,...,TCGA-BH-A0DT-11A,TCGA-E9-A1R0-01A,TCGA-BH-A0B6-01A,TCGA-B6-A0RN-01A,TCGA-A8-A09W-01A,TCGA-EW-A1P3-01A,TCGA-A7-A13F-11A,TCGA-A2-A0T6-01A,TCGA-A7-A5ZW-01A,TCGA-BH-A203-01A
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000242268.2,0.091708,0.000000,0.057899,0.000000,0.000000,0.354334,0.000000,0.000000,0.000000,0.029266,...,0.234169,0.107850,0.000000,0.000000,0.000000,0.000000,0.471630,0.083825,0.000000,0.061585
ENSG00000270112.3,0.019573,0.004701,0.016302,0.000000,0.000000,0.000000,0.005787,0.000000,0.000000,0.028433,...,0.000000,0.003879,0.010047,0.000000,0.010919,0.000000,0.000000,0.000000,0.005858,0.008704
ENSG00000167578.15,2.235898,1.863334,1.704753,1.947481,2.734690,2.397119,2.337327,2.256976,1.968791,2.240375,...,1.928209,2.004722,2.971522,1.841232,1.185122,2.861514,1.626213,1.692995,2.051916,2.197365
ENSG00000273842.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000078237.5,2.321945,4.226699,1.975755,2.808757,1.964479,1.792870,1.939951,2.681632,2.091932,1.919117,...,2.024599,2.216023,2.124097,1.665709,2.217943,2.120565,1.605675,2.055448,2.195782,3.291072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000105063.17,3.680055,3.969785,3.011921,4.059347,4.249147,3.935869,4.082827,3.931459,3.870035,4.266755,...,3.348868,3.650676,4.478325,3.520017,3.372176,3.772585,3.133380,4.186347,3.340223,3.966532
ENSG00000231119.2,0.285640,0.115149,0.384451,0.345883,0.065679,0.837305,0.198863,0.216728,0.102007,0.164329,...,0.329875,0.232873,0.069587,0.117933,0.343273,0.149851,0.270187,0.484561,0.127070,0.174248
ENSG00000280861.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
reads2 = reads.reset_index()
reads2.rename(columns={'Ensembl_ID':'sample_ID'}, inplace=True)
reads2

,sample_ID,TCGA-E9-A1NI-01A,TCGA-A1-A0SP-01A,TCGA-BH-A1EU-11A,TCGA-A8-A06X-01A,TCGA-E2-A14T-01A,TCGA-AC-A8OS-01A,TCGA-A8-A09K-01A,TCGA-OL-A5RY-01A,TCGA-BH-A0DG-01A,...,TCGA-BH-A0DT-11A,TCGA-E9-A1R0-01A,TCGA-BH-A0B6-01A,TCGA-B6-A0RN-01A,TCGA-A8-A09W-01A,TCGA-EW-A1P3-01A,TCGA-A7-A13F-11A,TCGA-A2-A0T6-01A,TCGA-A7-A5ZW-01A,TCGA-BH-A203-01A
0,ENSG00000242268.2,0.091708,0.000000,0.057899,0.000000,0.000000,0.354334,0.000000,0.000000,0.000000,...,0.234169,0.107850,0.000000,0.000000,0.000000,0.000000,0.471630,0.083825,0.000000,0.061585
1,ENSG00000270112.3,0.019573,0.004701,0.016302,0.000000,0.000000,0.000000,0.005787,0.000000,0.000000,...,0.000000,0.003879,0.010047,0.000000,0.010919,0.000000,0.000000,0.000000,0.005858,0.008704
2,ENSG00000167578.15,2.235898,1.863334,1.704753,1.947481,2.734690,2.397119,2.337327,2.256976,1.968791,...,1.928209,2.004722,2.971522,1.841232,1.185122,2.861514,1.626213,1.692995,2.051916,2.197365
3,ENSG00000273842.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSG00000078237.5,2.321945,4.226699,1.975755,2.808757,1.964479,1.792870,1.939951,2.681632,2.091932,...,2.024599,2.216023,2.124097,1.665709,2.217943,2.120565,1.605675,2.055448,2.195782,3.291072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60478,ENSG00000105063.17,3.680055,3.969785,3.011921,4.059347,4.249147,3.935869,4.082827,3.931459,3.870035,...,3.348868,3.650676,4.478325,3.520017,3.372176,3.772585,3.133380,4.186347,3.340223,3.966532
60479,ENSG00000231119.2,0.285640,0.115149,0.384451,0.345883,0.065679,0.837305,0.198863,0.216728,0.102007,...,0.329875,0.232873,0.069587,0.117933,0.343273,0.149851,0.270187,0.484561,0.127070,0.174248
60480,ENSG00000280861.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60481,ENSG00000123685.7,0.599579,1.382192,0.629043,0.396315,0.157504,0.662529,0.174878,1.724989,0.580636,...,0.487383,0.491877,1.261285,0.407713,0.379536,0.551165,0.611319,0.706170,0.683710,0.430933


In [10]:
label = pd.read_csv('TCGA-BRCA.GDC_phenotype.tsv', sep='\t', header=0, index_col=0)
label

,additional_pharmaceutical_therapy,additional_radiation_therapy,additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,age_at_initial_pathologic_diagnosis,axillary_lymph_node_stage_method_type,axillary_lymph_node_stage_other_method_descriptive_text,batch_number,bcr,bcr_followup_barcode,...,days_to_collection.samples,days_to_sample_procurement.samples,initial_weight.samples,is_ffpe.samples,oct_embedded.samples,preservation_method.samples,sample_type.samples,sample_type_id.samples,state.samples,tissue_type.samples
submitter_id.samples,,,,,,,,,,,,,,,,,,,,,
TCGA-A2-A0CY-01A,NaN,NaN,NaN,NaN,63.0,Sentinel lymph node biopsy plus axillary disse...,NaN,56.91.0,Nationwide Children's Hospital,TCGA-A2-A0CY-F13731,...,1274.0,NaN,220.0,False,true,NaN,Primary Tumor,1.0,released,Not Reported
TCGA-B6-A40B-01A,NaN,NaN,NaN,NaN,76.0,Sentinel node biopsy alone,NaN,239.56.0,Nationwide Children's Hospital,TCGA-B6-A40B-F59661,...,2467.0,NaN,10.0,False,true,NaN,Primary Tumor,1.0,released,Not Reported
TCGA-AO-A0J8-01A,NaN,NaN,NaN,NaN,61.0,Sentinel node biopsy alone,NaN,56.91.0,Nationwide Children's Hospital,TCGA-AO-A0J8-F15482,...,241.0,NaN,670.0,False,true,NaN,Primary Tumor,1.0,released,Not Reported
TCGA-A8-A08J-01A,NaN,NaN,NaN,NaN,52.0,NaN,NaN,47.98.0,Nationwide Children's Hospital,TCGA-A8-A08J-F40846,...,1733.0,NaN,630.0,False,false,NaN,Primary Tumor,1.0,released,Not Reported
TCGA-E2-A14N-01A,NaN,NaN,NaN,NaN,37.0,Sentinel lymph node biopsy plus axillary disse...,NaN,103.79.0,Nationwide Children's Hospital,TCGA-E2-A14N-F19154,...,1240.0,NaN,440.0,False,false,NaN,Primary Tumor,1.0,released,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-E2-A10B-01A,YES,YES,YES,NaN,67.0,Sentinel lymph node biopsy plus axillary disse...,NaN,80.84.0,Nationwide Children's Hospital,TCGA-E2-A10B-F17370,...,1065.0,NaN,210.0,False,false,NaN,Primary Tumor,1.0,released,Not Reported
TCGA-BH-A0E0-01A,NaN,NaN,NaN,NaN,38.0,Axillary lymph node dissection alone,NaN,61.89.0,Nationwide Children's Hospital,TCGA-BH-A0E0-F4603,...,136.0,NaN,620.0,False,true,NaN,Primary Tumor,1.0,released,Not Reported
TCGA-BH-A0E0-11A,NaN,NaN,NaN,NaN,38.0,Axillary lymph node dissection alone,NaN,61.89.0,Nationwide Children's Hospital,TCGA-BH-A0E0-F4603,...,136.0,NaN,140.0,False,true,NaN,Solid Tissue Normal,11.0,released,Not Reported
